In [42]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
import datetime as dt

## Chargement des fichiers source

In [43]:
UO_ref = pd.read_excel(r"C:\Users\sgasmi\Desktop\Export Python\Referentiel UO.xlsx", sheet_name="Référentiel UO")
UO_aff = pd.read_excel(r"C:\Users\sgasmi\Desktop\Export Python\Referentiel UO.xlsx", sheet_name="Affectation des salariés")
sal = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str})
loc_excel =r"C:\Users\sgasmi\Desktop\Template\Template UO.xlsx"

### Transformation des fichiers source

In [44]:
UO_aff["Salarié"] = UO_aff.apply(lambda x : x["Nom"] + " " + x["Prénom"] + " (" + str(x["Matricule"]) + ")", axis=1)

In [45]:
cols = ["Salarié", 'Matricule', 'Nom', 'Prénom', 'BU', 'Code UT', 'UO',
       "Nom de l'ent. juridique", 'Matricule Manager Direct',
       'Nom Manager Direct', 'Prénom Manager Direct',
       'Matricule responsable UO', 'Nom responsable UO',
       'Prénom responsabe UO', 'Matricule RRH Direct', 'Nom RRH Direct',
       'Prénom RRH Direct', "Matricule RRH d'UO", "Prénom RRH d'UO",
       "Nom RRH d'UO"]
UO_aff = UO_aff[cols]

In [46]:
UO_aff = UO_aff.sort_values(["Salarié"])

In [47]:
UO_ref = UO_ref[["Nom de l'unité organisationnelle","Code de l'unité organisationnelle",
       'Niveau', 'Description', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8',
       'BU', 'Code unité de tr. mère', 'UO mère', "Code de l'unité de travail",
       'Matricule Manager', 'Nom Manager', 'Prénom Manager', 'Matricule RRH',
       'Nom RRH', 'Prénom RRH','Nb salarié(s)', 'Nb enfants', 'Manager vide',
       'RRH vide', 'Date mise à jour']]

In [48]:
UO_ref = UO_ref.sort_values(["Nom de l'unité organisationnelle"])

In [49]:
sal = sal[sal["Code Contrat"].isin(["STA2", "STA1"])==False].reset_index(drop=True)
sal = sal[["Nom", "Prénom","Matricule"]]
sal["Salarié"] = sal.apply(lambda x: x["Nom"] + " " + x["Prénom"] + " (" + str(x["Matricule"]) + ")", axis=1)
sal = sal[['Salarié', 'Nom', 'Prénom', 'Matricule']]
sal = sal.sort_values(["Salarié"])

## Ajout des onglets actualisés dans le template

In [50]:
wb = load_workbook(filename = loc_excel)

In [51]:
sheet_to_delete = ["UO", "Affectation salariés", "Liste salariés"]

for sheet in sheet_to_delete:
    wb.remove(wb[sheet])
  
wb.save(loc_excel)

In [52]:
with pd.ExcelWriter(loc_excel, engine='openpyxl') as writer:
    writer.book = wb
    writer.sheets = {
        worksheet.title: worksheet
        for worksheet in wb.worksheets
    }
    UO_ref.to_excel(writer, sheet_name = 'UO', index=False)
    UO_aff.to_excel(writer, sheet_name = 'Affectation salariés', index=False)
    sal.to_excel(writer, sheet_name = 'Liste salariés', index=False)
    writer.save()

### Ajout date de version

In [53]:
today = dt.date.today()
date_jour = today.strftime('%d %m %Y')
date_jour = date_jour.replace(" ", "/")

In [54]:
sheet_UO = wb["UO"]
sheet_UO.sheet_state = 'hidden'
sheet_aff = wb["Affectation salariés"]
sheet_aff.sheet_state = 'hidden'
sheet_sal = wb["Liste salariés"]
sheet_sal.sheet_state = 'hidden'

In [55]:
sheet = wb['Documentation']

In [56]:
sheet["A1"] = "Version du " + str(date_jour)
wb.save(loc_excel)
wb.close()